In [55]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

In [56]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-03-06 01:18:49--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  6.15MB/s    in 0.2s    

2022-03-06 01:18:49 (6.15 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [57]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [58]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...| 2015-08-31|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          Beauty|          5|

In [59]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_df = df.select(["customer_id", "review_id", "product_id", "product_parent",
                       "product_title", "star_rating", "helpful_votes", "total_votes",
                       "vine", "verified_purchase", "review_date"])
review_df.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+-----------+
|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          5|            0|          0|   N|                Y| 2015-08-31|
|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          5|            0|          0|   N|                Y| 2015-08-31|
|   19242472|R3LIDG2Q4LJBAO|B00HU6UQAG|     375449471|Elysee Infusion S...|          5|            0|          0|   N|                Y| 2015-08-31|
|   19551372|R3KSZHPAEVPEAL|B002HWS7RM|     255651889|Diane D722 Color,...|          5|            0|     

In [60]:
review_df.count()
review_df.dropna()
review_df.dropDuplicates()
review_df.count()

5115666

In [61]:
# Create the vine_table. DataFrame
from pyspark.sql.types import IntegerType
review_df = review_df.withColumn("star_rating", review_df["star_rating"].cast(IntegerType()))
vine_df = review_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3I2DHQBR577SS|          5|            0|          0|   N|                Y|
|R1QNE9NQFJC2Y4|          5|            0|          0|   N|                Y|
|R3LIDG2Q4LJBAO|          5|            0|          0|   N|                Y|
|R3KSZHPAEVPEAL|          5|            0|          0|   N|                Y|
| RAI2OIG50KZ43|          5|            0|          0|   N|                Y|
|R1R30FA4RB5P54|          4|            0|          0|   N|                Y|
|R30IJKCGJBGPJH|          5|            0|          0|   N|                Y|
|R18GLJJPVQ1OVH|          5|            0|          0|   N|                Y|
| R8TVYIJXLYJT0|          5|            0|          0|   N|                Y|
|R1CJGF6M3PVHEZ|          1|            0|          2|   N|     

In [62]:
# Filter the data and create a new DataFrame to retrieve all rows where 
# total_votes count is equal to or greater than 20
total_votes_df = vine_df.filter("total_votes >= 20").select(['review_id','star_rating','helpful_votes',
                                                            'total_votes','vine','verified_purchase'])
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|
| RZN43XRZ89IIJ|          3|           10|         27|   N|                N|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y|
|R17ARFSDV555EW|          1|           29|         29|   N|                Y|
|R3GL5156FFEDQA|          5|           20|         21|   N|                Y|
|R29KQJC9PIYWQS|          5|           25|         26|   N|     

In [63]:
# Filter DataFrame created in Step 1 and create a new DataFrame to retrieve all 
# the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
helpful_votes_df = total_votes_df
helpful_votes_df = helpful_votes_df.withColumn('percentage', helpful_votes_df['helpful_votes']/helpful_votes_df['total_votes']*100)
helpful_votes_df.filter("percentage >= 50").select(['review_id','star_rating','helpful_votes',
                                                            'total_votes','vine','verified_purchase','percentage'])
helpful_votes_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       percentage|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|78.26086956521739|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|83.33333333333334|
| RZN43XRZ89IIJ|          3|           10|         27|   N|                N|37.03703703703704|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|95.83333333333334|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|             90.0|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|97.36842105263158|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y| 97.1830985915493|
|R17ARFSDV555EW|          1|           2

In [64]:
# Filter the DataFrame created in Step 2, and create a new DataFrame that retrieves all 
# the rows where a review was written as part of the Vine program (paid), vine == 'Y'
paid_df = helpful_votes_df.filter("vine == 'Y'")
paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       percentage|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R106V6GUNWRSSS|          5|          857|        889|   Y|                N|96.40044994375702|
|R37N8KZS48W36A|          5|           36|         37|   Y|                N| 97.2972972972973|
|R1LNEQ2MQE03PD|          5|           49|         66|   Y|                N|74.24242424242425|
|R1ECPWPUNK36ES|          5|           29|         31|   Y|                N|93.54838709677419|
|R16BUM7UQZLOM6|          3|          107|        112|   Y|                N|95.53571428571429|
| R7NWIOCA5RVPR|          5|           19|         26|   Y|                N|73.07692307692307|
|R3NTGOCUZB33JK|          5|           40|         47|   Y|                Y| 85.1063829787234|
| RQX3RHGMG7UBN|          5|          18

In [65]:
# Repeat previous step except with vine == 'N'
unpaid_df = helpful_votes_df.filter("vine == 'N'")
unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       percentage|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|78.26086956521739|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|83.33333333333334|
| RZN43XRZ89IIJ|          3|           10|         27|   N|                N|37.03703703703704|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|95.83333333333334|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|             90.0|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|97.36842105263158|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y| 97.1830985915493|
|R17ARFSDV555EW|          1|           2

In [80]:
# total number of reviews, number of 5-star reviews, and the percentage of 
# 5-star reviews are calculated for all Vine and non-Vine reviews
from pyspark.sql.functions import count, sum
total_paidReviews = paid_df.groupby("review_id").agg(count("review_id"))
total_paid = total_paidReviews.count()
total_paid

658

In [82]:
total_unpaidReviews = unpaid_df.groupby("review_id").agg(count("review_id"))
total_unpaid = total_unpaidReviews.count()
total_unpaid

78569

In [67]:
total_fiveStars = helpful_votes_df.filter("star_rating==5").count()
total_fiveStars

44064

In [77]:
paid_star = paid_df.filter("star_rating==5").count()
# paid_star_percent = paid_df.withColumn('star_percent', paid_df.filter('star_rating=='5'')/total_fiveStars * 100)
paid_star

229

In [81]:
paid_star_percent = paid_star / total_paid * 100
paid_star_percent 

34.80243161094225

In [70]:
unpaid_star = unpaid_df.filter("star_rating==5").count()
unpaid_star

43835

In [83]:
unpaid_star_percent = unpaid_star / total_unpaid * 100
unpaid_star_percent 

55.79172447148366